In [2]:
import os
import pandas as pd
import shutil

import torch
import torchio as tio

In [3]:
SRC_PATH = "../datasets/cmrxmotion/train/"
DST_PATH = "../datasets/cmrxmotion/train_aug/"
os.makedirs(DST_PATH, exist_ok=True)

src_path_data = os.path.join(SRC_PATH, "data")
dst_path_data = os.path.join(DST_PATH, "data")
os.makedirs(dst_path_data, exist_ok=True)

src_label_info = os.path.join(SRC_PATH, "IQA.csv")
dst_label_info = os.path.join(DST_PATH, "IQA.csv")
shutil.copy(src_label_info, dst_label_info)

'../datasets/cmrxmotion/train_aug/IQA.csv'

In [4]:
# Transfer original files from SRC to DST
for case in sorted(os.listdir(src_path_data)):
    src_case_path = os.path.join(src_path_data, case)
    dst_case_path = os.path.join(dst_path_data, case)
    os.makedirs(dst_case_path, exist_ok=True)

    for file in sorted(os.listdir(src_case_path)):
        src_file_path = os.path.join(src_case_path, file)
        dst_file_path = os.path.join(dst_case_path, file)
        if "label" in file:
            shutil.copy(src_file_path, dst_file_path)
        else:
            shutil.copy(src_file_path, dst_file_path)

In [15]:
"""
This block performs the following actions:
    1) Reads the nifti files in SRC
    2) Applies motion artefact or some other class with a similar disturbing effect on the nifti file
    3) Rescales the new nifti file based on the min-max values of the original image
    4) Appends to the IQA.csv file
    5) Saves the new nifti file into the DST folder

Notes:
    1) This block assumes that the file/folder structure is already identical to the SRC dataset.
"""
# Generate moderate-level artefacts by using good-quality images
df = pd.read_csv(src_label_info)
dict_aug = {"Image": [], "Label": []}

augment = tio.transforms.RandomMotion(degrees=(0.2, 0.25),
                                      translation=0.,
                                      num_transforms=2,
                                    #   image_interpolation="lanczos"
                                      image_interpolation="linear"
                                      )

for image, label in zip(df["Image"], df["Label"]):
    if label == 1:
        pid, mode, phase = image.split('-')

        src_case_path = os.path.join(src_path_data, f"{pid}-{mode}")
        dst_case_path = os.path.join(dst_path_data, f"{pid}-{mode}_aug2")
        os.makedirs(dst_case_path, exist_ok=True)
        
        src_file_path = os.path.join(src_case_path, f"{pid}-{mode}-{phase}.nii.gz")
        dst_file_path = os.path.join(dst_case_path, f"{pid}-{mode}_aug2-{phase}.nii.gz")
        rescl_dst_file_path = os.path.join(dst_case_path, f"{pid}-{mode}_aug2_rescl-{phase}.nii.gz")
        src_label_file_path = os.path.join(src_case_path, f"{pid}-{mode}-{phase}-label.nii.gz")
        dst_label_file_path = os.path.join(dst_case_path, f"{pid}-{mode}_aug2-{phase}-label.nii.gz")
    
        cmr_image = tio.ScalarImage(src_file_path)
        # max_val, min_val = cmr_image.data.max().item(), cmr_image.data.min().item()
        # max_val, min_val = torch.quantile(cmr_image.data, torch.Tensor([0.997, 0.003]), interpolation='nearest')
        augmented_cmr_image = augment(cmr_image)
        max_val, min_val = augmented_cmr_image.data.max().item(), augmented_cmr_image.data.min().item()
        rescaler = tio.RescaleIntensity(out_min_max=(0, max_val), percentiles=(1.0, 99.9))
        rescaled_augmented_cmr_image = rescaler(augmented_cmr_image)

        augmented_cmr_image.save(dst_file_path)
        rescaled_augmented_cmr_image.save(rescl_dst_file_path)
        shutil.copy(src_label_file_path, dst_label_file_path)

        dict_aug["Image"].append(f"{pid}-{mode}_aug2-{phase}")
        dict_aug["Label"].append(2)
        
df_aug = pd.DataFrame(dict_aug)
df = pd.concat([df, df_aug], ignore_index=True)
df.to_csv(dst_label_info, index=False)
print(df)


In [ ]:
"""
This block performs the following actions:
    1) Reads the nifti files in SRC
    2) Applies motion artefact or some other class with a similar disturbing effect on the nifti file
    3) Rescales the new nifti file based on the min-max values of the original image
    4) Appends to the IQA.csv file
    5) Saves the new nifti file into the DST folder

Notes:
    1) This block assumes that the file/folder structure is already identical to the SRC dataset.
"""
# Generate moderate-level artefacts by using good-quality images
df = pd.read_csv(dst_label_info)
dict_aug = {"Image": [], "Label": []}

augment = tio.transforms.RandomMotion(degrees=(0.45, 0.55),
                                      translation=0.,
                                      num_transforms=2,
                                    #   image_interpolation="lanczos"
                                      image_interpolation="linear"
                                      )

for image, label in zip(df["Image"], df["Label"]):
    if label == 1:
        pid, mode, phase = image.split('-')

        src_case_path = os.path.join(src_path_data, f"{pid}-{mode}")
        dst_case_path = os.path.join(dst_path_data, f"{pid}-{mode}_aug3")
        os.makedirs(dst_case_path, exist_ok=True)
        
        src_file_path = os.path.join(src_case_path, f"{pid}-{mode}-{phase}.nii.gz")
        dst_file_path = os.path.join(dst_case_path, f"{pid}-{mode}_aug3-{phase}.nii.gz")
        rescl_dst_file_path = os.path.join(dst_case_path, f"{pid}-{mode}_aug3_rescl-{phase}.nii.gz")
        src_label_file_path = os.path.join(src_case_path, f"{pid}-{mode}-{phase}-label.nii.gz")
        dst_label_file_path = os.path.join(dst_case_path, f"{pid}-{mode}_aug3-{phase}-label.nii.gz")
    
        cmr_image = tio.ScalarImage(src_file_path)
        # max_val, min_val = cmr_image.data.max().item(), cmr_image.data.min().item()
        # max_val, min_val = torch.quantile(cmr_image.data, torch.Tensor([0.997, 0.003]), interpolation='nearest')
        augmented_cmr_image = augment(cmr_image)
        max_val, min_val = augmented_cmr_image.data.max().item(), augmented_cmr_image.data.min().item()
        rescaler = tio.RescaleIntensity(out_min_max=(0, max_val), percentiles=(1.0, 99.9))
        rescaled_augmented_cmr_image = rescaler(augmented_cmr_image)

        augmented_cmr_image.save(dst_file_path)
        rescaled_augmented_cmr_image.save(rescl_dst_file_path)
        shutil.copy(src_label_file_path, dst_label_file_path)

        dict_aug["Image"].append(f"{pid}-{mode}_aug3-{phase}")
        dict_aug["Label"].append(3)
        
df_aug = pd.DataFrame(dict_aug)
df = pd.concat([df, df_aug], ignore_index=True)
df.to_csv(dst_label_info, index=False)
print(df)


              Image  Label
0         P001-1-ED      1
1         P001-2-ED      2
2         P001-3-ED      2
3         P001-4-ED      1
4         P001-1-ES      1
..              ...    ...
225  P019-4_aug3-ED      3
226  P019-1_aug3-ES      3
227  P019-4_aug3-ES      3
228  P020-1_aug3-ED      3
229  P020-1_aug3-ES      3

[230 rows x 2 columns]
